In [2]:
import numpy as np
import torch
import torch.nn as nn

In [82]:
datax = [
    [[0.1517, 0.5443, 0.1664, 0.2968, 0.1047, 0.2303, 0.7809, 0.8742]],
    [[0.6482, 0.8415, 0.3437, 0.5621, 0.8318, 0.5978, 0.2179, 0.5826]],
    [[0.0959, 0.5683, 0.8281, 0.2220, 0.7972, 0.3602, 0.9269, 0.3220]],
    [[0.2017, 0.4027, 0.9019, 0.3093, 0.2855, 0.0226, 0.3115, 0.5141]],
]


# data shape
batch = 4
length = 8

# conv shape
k_size = 3
chan_in = 1
chan_out = 2

params = np.random.rand(k_size, chan_in, chan_out)


AttributeError: 'list' object has no attribute 'size'

In [83]:
np_input = np.array(datax).reshape(batch, length, chan_in)
print(np_input.shape)
th_input = torch.Tensor(datax)

(4, 8, 1)


In [80]:
from lamp.conv import Conv1D

m1 = Conv1D(k_size, chan_in, chan_out)
output = m1.forward(np_input)
print(output.shape)
np.round_(output.reshape(batch, chan_out, -1), decimals=4)

(4, 6, 2)


array([[[ 0.0463,  0.0321,  0.0064, -0.0147,  0.023 ,  0.0132],
        [ 0.0071, -0.0065,  0.0442,  0.0174,  0.0893,  0.0508]],

       [[ 0.0609,  0.0283,  0.0208, -0.0157,  0.0657,  0.0287],
        [ 0.0722,  0.0344,  0.0298,  0.    ,  0.0193, -0.0115]],

       [[ 0.0744,  0.0424,  0.058 ,  0.0306,  0.0198, -0.0217],
        [ 0.0721,  0.0481,  0.0371, -0.0091,  0.0769,  0.0496]],

       [[ 0.0597,  0.0255,  0.0729,  0.0455,  0.0058, -0.0238],
        [ 0.0142,  0.004 ,  0.0037, -0.011 ,  0.044 ,  0.0254]]])

In [78]:
m2 = nn.Conv1d(chan_in, chan_out, k_size)
output = m2(th_input)
output.shape

torch.Size([4, 2, 6])

In [7]:
from lamp import MaxPool1D

m3 = MaxPool1D(k_size, stride=1)
output = m3.forward(np_input)
output.reshape(batch, chan_in, -1)

array([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

       [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]],

       [[0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269]],

       [[0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]])

In [42]:
m4 = nn.MaxPool1d(k_size, stride=1)
output = m4(th_input)
output

tensor([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

        [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]],

        [[0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269]],

        [[0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]])

In [9]:
input = m4(th_input)
# With default parameters
m6 = nn.Flatten()
output = m6(input)
output.size(), output

(torch.Size([4, 6]),
 tensor([[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742],
         [0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978],
         [0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269],
         [0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]))

In [10]:
from lamp import Flatten

input = m3.forward(np_input)

m5 = Flatten()
output = m5.forward(input)
output.shape, output

((4, 6),
 array([[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742],
        [0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978],
        [0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269],
        [0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]))

In [14]:
stride = 1
from lamp.conv import Conv1D

m1 = Conv1D(k_size, chan_in, chan_out)
output = m1.forward(np_input)
np.round_(output.reshape(batch, chan_out, -1), decimals=4)

from lamp import MaxPool1D

m3 = MaxPool1D(k_size, stride=1)
output2 = m3.forward(np_input)
output2.reshape(batch, chan_in, -1)


array([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

       [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]],

       [[0.8281, 0.8281, 0.8281, 0.7972, 0.9269, 0.9269]],

       [[0.9019, 0.9019, 0.9019, 0.3093, 0.3115, 0.5141]]])

In [ ]:
for k in range(d_out):
            t1, t2 = k * self.stride, 2 * (self.k_size // 2) + k * self.stride + 1
            res[:, k, :] = np.sum(
                X[:, t1:t2, :, np.newaxis] * self._parameters[np.newaxis, :, :, :],
                axis=(1, 2),
            )


 for i in range(d_out):
            out[:,i, :] = np.sum(X[:,self.k_size//2+i*self.stride -self.k_size//2:self.k_size//2+i*self.stride +self.k_size//2+1, :, np.newaxis] * self._parameters[np.newaxis, :, :, : ],axis = (1,2))

In [171]:
li = np.arange(8)
print(li)
print(li[0:-7])

[0 1 2 3 4 5 6 7]
[0]


In [172]:
m1 = Conv1D(k_size, chan_in, chan_out)
output = m1.forward(np_input)

delta = output
print(output, k_size)
d_out = (length - k_size) // stride + 1

d_out = 8
res = np.zeros((batch, d_out, chan_in))
for k in range(d_out):
    t1, t2 = 0, k_size 
    d1 = - k * stride - 2 + k_size
    d2 = - (k * stride + 2)
  
    if k == 0 :
       
        t1 = 0
        t2 = k + 1
        d1 = None
        d2 = - 1
    elif k <= k_size -1 :
        
        t1 = 0
        t2 = k + 1
        d1 = - 1
        d2 = - k -2
    elif k > d_out - k_size -1 :
        
        t1 = k_size - (d_out - k)
        t2 = k_size
        d1 = - k +2
        d2 = 0
   
 
    res[:, k, :] = np.sum(
        np.flip(params, axis=1)[ np.newaxis, t1:t2, :,  :]
        * delta[  :, d2:d1, np.newaxis, :],
        axis=(1,3)
    )
    print(res[:, k, :])
 
print(res)

[[[-0.0517664   0.02651739]
  [ 0.00028854 -0.01245236]
  [-0.02323057  0.00977333]
  [-0.00682623 -0.00297487]
  [-0.07834322  0.0406105 ]
  [-0.12648592  0.06582866]]

 [[-0.05678573  0.01861336]
  [-0.01597467 -0.01005455]
  [-0.0948778   0.04519597]
  [-0.08350796  0.03500214]
  [-0.01078753 -0.01154108]
  [-0.02431258 -0.00028029]]

 [[-0.11338295  0.06109375]
  [-0.0512864   0.01750703]
  [-0.0256639  -0.00503623]
  [-0.08557511  0.04409042]
  [-0.05123603  0.01003953]
  [-0.08388056  0.04042555]]

 [[-0.09684531  0.04897313]
  [-0.0774355   0.0358384 ]
  [ 0.01470813 -0.0281459 ]
  [-0.00481815 -0.00359083]
  [-0.00705548 -0.00292802]
  [-0.06939058  0.03809829]]] 3
k 0 3
0 3
-2 1
k == 0
0 1
-1 None
delta [[[[-0.12648592  0.06582866]]]


 [[[-0.02431258 -0.00028029]]]


 [[[-0.08388056  0.04042555]]]


 [[[-0.06939058  0.03809829]]]]
ari [[[[0.08315117 0.65322209]]]]
[[ 0.03248329]
 [-0.00220471]
 [ 0.01943209]
 [ 0.01911674]]
[[[-0.0517664   0.02651739]
  [ 0.00028854 -0.012452